In [ ]:
# Optuna import 추가
import optuna
from optuna.samplers import TPESampler

def objective(trial, x, y, building_type):
    """
    Optuna objective function for LightGBM hyperparameter optimization
    """
    # 파라미터 제안
    params = {
        'learning_rate': 0.05,  # 고정
        'n_estimators': 5000,   # 고정 (early stopping 사용)
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'alpha': trial.suggest_float('alpha', 0.5, 3.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 2.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 2.0),
        'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0.0, 1.0),
        'random_state': RANDOM_SEED,
        'device': 'gpu',
        'verbosity': -1
    }
    
    # 교차검증
    kf_opt = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
    smape_scores = []
    
    for train_idx, val_idx in kf_opt.split(x):
        X_train, X_val = x.iloc[train_idx], x.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        # log1p 변환
        y_train_t = np.log1p(np.clip(y_train, a_min=0, a_max=None))
        y_val_t = np.log1p(np.clip(y_val, a_min=0, a_max=None))
        
        model = LGBMRegressor(
            learning_rate=params['learning_rate'],
            n_estimators=params['n_estimators'],
            max_depth=params['max_depth'],
            subsample=params['subsample'],
            feature_fraction=params['feature_fraction'],
            min_child_samples=params['min_child_samples'],
            reg_alpha=params['reg_alpha'],
            reg_lambda=params['reg_lambda'],
            min_gain_to_split=params['min_gain_to_split'],
            objective=weighted_mse_lgb(params['alpha']),
            random_state=params['random_state'],
            device=params['device'],
            verbosity=params['verbosity']
        )
        
        model.fit(
            X_train, y_train_t,
            eval_set=[(X_val, y_val_t)],
            callbacks=[lightgbm.early_stopping(50), lightgbm.log_evaluation(0)]
        )
        
        # 예측 및 SMAPE 계산
        pred_t = model.predict(X_val)
        pred = np.expm1(pred_t)
        score = smape(y_val.values, pred)
        smape_scores.append(score)
    
    return np.mean(smape_scores)

# 건물 타입별 최적화 실행
def optimize_parameters():
    """
    각 건물 타입별로 파라미터 최적화 수행
    """
    best_params_dict = {}
    
    print("=" * 60)
    print("Optuna를 이용한 건물 타입별 하이퍼파라미터 최적화 시작")
    print("=" * 60)
    
    for i, building_type in enumerate(type_list):
        print(f"\n[{i+1}/{len(type_list)}] 건물 타입: {building_type} 최적화 중...")
        
        # 데이터 준비
        x = X[X.building_type == building_type].copy()
        y = Y[Y.building_type == building_type]['power_consumption'].astype(float).copy()
        
        # 라벨 클린업
        y = y.replace([np.inf, -np.inf], np.nan)
        ok = y.notna() & (y >= 0)
        if not ok.all():
            x = x.loc[ok]
            y = y.loc[ok]
        
        # 원핫 인코딩
        x_encoded = pd.get_dummies(x, columns=['building_number'], drop_first=False)
        
        # building_type 컬럼 제거
        if 'building_type' in x_encoded.columns:
            x_encoded.drop(columns=['building_type'], inplace=True)
        
        # NaN/Inf 처리
        x_encoded = x_encoded.replace([np.inf, -np.inf], np.nan).fillna(0)
        x_encoded = x_encoded.astype(np.float32)
        y = y.astype(np.float32)
        
        # 데이터가 너무 적으면 스킵
        if len(x_encoded) < 100:
            print(f"  데이터가 부족합니다 ({len(x_encoded)}개). 기본 파라미터 사용.")
            best_params_dict[building_type] = {
                'max_depth': 8,
                'subsample': 0.8,
                'feature_fraction': 0.8,
                'min_child_samples': 20,
                'alpha': 1.5,
                'reg_alpha': 0.1,
                'reg_lambda': 0.1,
                'min_gain_to_split': 0.0
            }
            continue
        
        # Optuna 최적화
        sampler = TPESampler(seed=RANDOM_SEED)
        study = optuna.create_study(direction='minimize', sampler=sampler)
        
        # 최적화 실행 (시간 단축을 위해 trials 수 조정)
        n_trials = 50 if len(x_encoded) > 1000 else 30
        
        try:
            study.optimize(
                lambda trial: objective(trial, x_encoded, y, building_type),
                n_trials=n_trials,
                timeout=1800,  # 30분 제한
                show_progress_bar=True
            )
            
            best_params = study.best_params
            best_score = study.best_value
            
            print(f"  최적 SMAPE: {best_score:.4f}")
            print(f"  최적 파라미터: {best_params}")
            
            best_params_dict[building_type] = best_params
            
        except Exception as e:
            print(f"  최적화 중 오류 발생: {e}")
            # 기본 파라미터 사용
            best_params_dict[building_type] = {
                'max_depth': 8,
                'subsample': 0.8,
                'feature_fraction': 0.8,
                'min_child_samples': 20,
                'alpha': 1.5,
                'reg_alpha': 0.1,
                'reg_lambda': 0.1,
                'min_gain_to_split': 0.0
            }
    
    # 결과를 DataFrame으로 변환하여 저장
    params_df = pd.DataFrame(best_params_dict).T
    params_df.reset_index(inplace=True)
    params_df.rename(columns={'index': 'building_type'}, inplace=True)
    params_df.to_csv('lgb_best_params_found.csv', index=False)
    
    print("\n" + "=" * 60)
    print("최적화 완료! 결과가 'lgb_best_params_found.csv'에 저장되었습니다.")
    print("=" * 60)
    
    return params_df

# 최적화 실행
if __name__ == "__main__":
    # 기존에 최적화된 파일이 있는지 확인
    import os
    if os.path.exists('lgb_best_params_found.csv'):
        print("기존 최적화 파일이 발견되었습니다.")
        use_existing = input("기존 파일을 사용하시겠습니까? (y/n): ")
        if use_existing.lower() != 'y':
            lgb_best_params = optimize_parameters()
        else:
            lgb_best_params = pd.read_csv('lgb_best_params_found.csv')
    else:
        lgb_best_params = optimize_parameters()
    
    # 인덱스 설정
    lgb_best_params.set_index('building_type', inplace=True)
    print("\n최적화된 파라미터:")
    print(lgb_best_params)